In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,KFold,train_test_split,cross_val_score,class


In [ ]:
data=pd.read_csv("glass.csv")

In [ ]:
data

In [ ]:
#the target variable is type of glass-Type
data.Type.value_counts()# we have 6 types of glass and corresponding number is given(4th type of glass label is not given)

In [ ]:
data.info()

In [ ]:
data.isna().sum()#there is null values.

In [ ]:
data.describe()

In [ ]:
# in problem we have given the full of all the features names-
#RI : refractive index
#Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
#Mg: Magnesium
#AI: Aluminum
#Si: Silicon
#K:Potassium
#Ca: Calcium
#Ba: Barium
#Fe: Iron
#we will edit the columns of features names
data=data.rename({"RI":"refractive_index","Na":"Sodium","Mg":"Magnesium","Al":"Aluminum","Si":"Silicon","K":"Potassium","Ca":"Calcium",
"Ba":"Barium","Fe":"Iron"},axis=1)


In [ ]:
data

In [ ]:
import seaborn as sns
sns.pairplot(data,hue="Type")

In [ ]:
sns.scatterplot(data['Aluminum'],data["Type"])# we can predict that aluminum is maximum in Type 7 class.

In [ ]:
import matplotlib.pyplot as plt
sns.swarmplot(data['Sodium'],data["Type"])

In [ ]:
sns.scatterplot(data['Magnesium'],data["Type"])#magnesium is slightly opposite to magnesium ,comparing type class.

In [ ]:
sns.scatterplot(data['Barium'],data["Type"])# type 7 has a higher barium values

In [ ]:
sns.scatterplot(data['Iron'],data["Type"])# type 1 and type 2 has maximum iron values,

In [ ]:
sns.scatterplot(data['Calcium'],data["Type"])# we can see that calcium values is not effecting the type catergory.

In [ ]:
sns.scatterplot(data['Potassium'],data["Type"])# we can see that potassium values is not effecting the type catergory.

In [ ]:
data1=data.drop(columns="Type")
data1

In [ ]:
from sklearn.preprocessing import StandardScaler
sf=StandardScaler()
data_standard=sf.fit_transform(data1)

In [ ]:
df_feat = pd.DataFrame(data_standard,columns=data.columns[:-1])
df_feat.head()

In [ ]:
df_feat=df_feat.drop(["Calcium","Potassium"],axis=1)#as from the graph the both features are not useful.

In [ ]:
#spliting the data in Train and test
X=df_feat
Y=data.iloc[:,9]
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3,random_state=20)


In [ ]:
X

In [ ]:

ytest

In [ ]:
#creating the model
kfold=KFold(n_splits=10)
knn=KNeighborsClassifier( metric='manhattan',
                    n_neighbors=4,
                     )
model=knn.fit(xtrain,ytrain)



In [ ]:
ypred=model.predict(xtest)

In [ ]:
print(classification_report(ytest,ypred))

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
accuracy_score(ytest,ypred)

In [ ]:
#Grid Search for Algorithm Tuning

In [ ]:
n_neighbors = np.array(range(1,40))
param_grid = dict(n_neighbors=n_neighbors)

In [ ]:
gd=GridSearchCV(estimator=KNeighborsClassifier(),param_grid=param_grid)
gd.fit(X,Y)

In [ ]:
model1=KNeighborsClassifier(n_neighbors=4)
result1=cross_val_score(model1,X,Y,cv=kfold)

In [ ]:
result1.mean()

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
# choose k between 1 to 41
k_range = range(1, 41)
k_scores = []
# use iteration to caclulator different k in models, then return the average accuracy based on the cross validation
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, Y, cv=5)
    k_scores.append(scores.mean())
# plot to see clearly
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()

In [ ]:
#best k value is 4

In [ ]:
#we will see the accuracy by bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
ac=AdaBoostClassifier(base_estimator=KNeighborsClassifier,n_estimators=10,random_state=40)
results = cross_val_score(ac, X, Y, cv=kfold)
results


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
estimators = []
model1 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))

# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X, Y, cv=kfold)
print(results.mean())